# ToF demo

This Jupyter notebook invokes the Vespucci jobcontrol-api to

- Train the model for ToF
- Benchmark the model and generate the network `.c`/`.h` sources
- Build the application firmware

### Initial setup

Dependencies are managed via the `poetry` command. You may install it with `pip install poetry`

Then, install all dependencies simply by invoking `poetry install` in this directory. This will create a virtual environment and place it in this folder as `.venv`.

Start a jupyter kernel in that environment and you're ready to run the notebook.

You will need API credentials for Vespucci. This example runs on vespucci-dev by default, so you should use a token generated for that environment.


In [ ]:
# Configuration preamble
import json
import httpx
import config
import utils
import asyncio

from collections.abc import Awaitable

client = httpx.Client(timeout=300, headers=utils.headers)


## Model training

This step will produce the `.h5` model artifact.

In [ ]:
with open("dataset", "rb") as dataset_file:
    print("Creating training job")
    job = client.post(
        config.JOBCONTROL_API_JOBS_ENDPOINT,
        files={
            "uploadedFile": dataset_file
        },
        data={
            "templateId": config.TRAINING_JOB_TEMPLATE_ID,
            "runtimeInput": json.dumps(
                {
                    "config": {
                        "dataset": {
                            "class_names": [
                                "One",
                                "Two",
                            ]
                        },
                        "stm32ai": {
                            "version": "8.1.0"
                        }
                    },
                }
            )
        }
    ).raise_for_status().json()


print(f"Created training job: {job!r}")

job_id = job["id"]

print("Blocking until job is done...")
job = await utils.block_until_done(job_id)

print("Training job is done. Downloading artifacts")
artifacts = utils.download_artifacts(job)

for key in artifacts:
    print(f"Writing {key}")
    with open(key, "wb") as file:
        file.write(artifacts[key])

# Ok for GC
del artifacts
client.delete(f"{config.JOBCONTROL_API_JOBS_ENDPOINT}/{job_id}")



## Benchmarking + Cube.AI

This step will generate `.c`/`.h` NN sources by invoking the STM32 Developer cloud APIs.

We'll download a zip containing said sources as well as the CubeAI runtime that will eventually be linked into the application firmware.

Simultaneously, we'll dispatch the benchmarkinging job and display the inference time once it's been measured.

In [ ]:
with open("best_model", "rb") as model_file:
    model = utils.NamedBuffer("model", model_file.read())

# devcloud API command
generate_command = {
    "command": "generate",
    "arguments":{
        "options": {
            "includeLibraryForSerie": "M4",
            "includeLibraryForIde": "gcc",
            "allocateInputs": True,
            "allocateOutputs": True,
            "compression": "none",
            "optimization": "balanced"
        },
    },
    "version": "8.1.0"
}

print("Creating Cube.AI job")
generate_job = client.post(
    config.JOBCONTROL_API_JOBS_ENDPOINT,
    files={
        "model": model,
    },
    data={
        "templateId": config.CUBEAI_JOB_TEMPLATE_ID,
        "runtimeInput": json.dumps({
            "command": generate_command
        })
    }
).raise_for_status().json()
print(f"Created CubeAI job: {generate_job!r}")

# devcloud API command
benchmarking_command = {
    "command": "benchmark",
    "arguments":{
        "options": {},
        "parameters": {
            "board_name": "NUCLEO-F401RE"
        }
    },
    "version": "8.1.0"
}

print("Creating benchmarking job")
benchmarking_job = client.post(
    config.JOBCONTROL_API_JOBS_ENDPOINT,
    files={
        "model": model,
    },
    data={
        "templateId": config.BENCHMARKING_JOB_TEMPLATE_ID,
        "runtimeInput": json.dumps({
            "command": benchmarking_command
        })
    }
).raise_for_status().json()
print(f"Created benchmarking job: {benchmarking_job!r}")


async def _generate_coro() -> Awaitable[None]:

    global generate_job

    job_id = generate_job["id"]

    generate_job = await utils.block_until_done(job_id)

    print("CubeAI job is done. Downloading artifacts")
    artifacts = utils.download_artifacts(generate_job)

    for key in artifacts:
        print(f"Writing {key}")
        with open(key, "wb") as file:
            file.write(artifacts[key])

    # Ok for GC
    del artifacts
    client.delete(f"{config.JOBCONTROL_API_JOBS_ENDPOINT}/{job_id}")


async def _benchmarking_coro() -> Awaitable[None]:

    global benchmarking_job

    job_id = benchmarking_job["id"]

    benchmarking_job = await utils.block_until_done(job_id)

    report = json.loads(benchmarking_job["outputs"]["parameters"]["benchmark_report"])
    print(f"Benchmarking job is done. Inference time: {report['duration_ms']}ms")


    client.delete(f"{config.JOBCONTROL_API_JOBS_ENDPOINT}/{job_id}")

await asyncio.wait((asyncio.create_task(_generate_coro()), asyncio.create_task(_benchmarking_coro())))

print("Cube.AI and benchmarking done.")


## Building

In this step we build the output NN sources into a deployment-ready application firmware.

In [ ]:
with open("output", "rb") as cubeai_output_file:
    cubeai_output = utils.NamedBuffer("network", cubeai_output_file.read())

print("Creating builder job")
job = client.post(
    config.JOBCONTROL_API_JOBS_ENDPOINT,
    files={
        "network": cubeai_output
    },
    data={
        "templateId": config.COMPILATION_JOB_TEMPLATE_ID,
        "runtimeInput": json.dumps({
            "config": {
                "dataset": {
                    "class_names": [
                        "One",
                        "Two",
                    ]
                },
                # "stm32ai": {
                #     "version": "8.1.0"
                # }
            },
        })
    }
).raise_for_status().json()

print(f"Created builder job: {job!r}")

job_id = job["id"]

print("Blocking until job is done...")
job = await utils.block_until_done(job_id)

print("Builder job is done. Downloading artifacts")
artifacts = utils.download_artifacts(job)

for key in artifacts:
    print(f"Writing {key}")
    with open(key, "wb") as file:
        file.write(artifacts[key])

# Ok for GC
del artifacts
client.delete(f"{config.JOBCONTROL_API_JOBS_ENDPOINT}/{job_id}")

In [ ]:
# Cleanup

client.close()